Python notebook based on https://towardsdatascience.com/fantasy-premier-league-value-analysis-python-tutorial-using-the-fpl-api-8031edfe9910#8ef2

Tutorial on how to get use the FPL API to gather data.

In [15]:
import requests
import pandas as pd
import numpy as np

In [16]:
# API url
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

In [17]:
# Use requests to get request from API
request = requests.get(url)

In [18]:
json = request.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

Building the dataframes we will use

In [19]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

For loop that loops through phases dictionary to see what it contains

for x in range(0,len(json['phases'])):
    for key,value in json['phases'][x].items():
        print(key,value)

In [20]:
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,NaN,80201,0,0,0,0,0,1,2.6,...,394,47,217,17,NaN,,NaN,,NaN,
1,0.0,NaN,115918,0,0,0,0,0,1,0.0,...,201,10,290,30,NaN,,NaN,,NaN,
2,50.0,NaN,47431,-1,1,-1,1,0,3,1.1,...,532,222,538,225,4.0,,2.0,,NaN,
3,50.0,25.0,54694,-1,1,-1,1,0,4,1.6,...,554,73,554,73,NaN,,NaN,,1.0,
4,NaN,NaN,58822,0,0,0,0,0,2,1.7,...,248,91,329,121,NaN,,5.0,,NaN,


Looking at all the columns of our dataframe

In [21]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [22]:
slim_elements_df = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

In [23]:
slim_elements_df.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points
0,Leno,1,1,2.0,50,90,2283,0.2,1
1,Rúnarsson,1,1,1.5,40,0,5640,0.0,0
2,Borges Da Silva,1,3,0.3,64,0,697,0.0,0
3,Aubameyang,1,4,1.5,99,0,928,0.0,0
4,Soares,1,2,0.2,45,0,501,0.0,0


In [24]:
slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

/var/folders/11/_0zmt03n4z77vx95_bc4kv2c0000gn/T/ipykernel_1220/1996727139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)


In [25]:
slim_elements_df.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position
0,Leno,1,1,2.0,50,90,2283,0.2,1,Goalkeeper
1,Rúnarsson,1,1,1.5,40,0,5640,0.0,0,Goalkeeper
2,Borges Da Silva,1,3,0.3,64,0,697,0.0,0,Midfielder
3,Aubameyang,1,4,1.5,99,0,928,0.0,0,Forward
4,Soares,1,2,0.2,45,0,501,0.0,0,Defender


In [26]:
filter = slim_elements_df.second_name == 'Ajer'
slim_elements_df[filter]

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position
85,Ajer,3,2,0.8,45,70,11575,1.3,6,Defender


In [36]:
date = input("day,month,year (spaces): ").split()
dated_filename = f'{date[2]}-{date[1]}-{date[0]}.csv'
elements_df.to_csv(dated_filename)

In [28]:
elements_df['code'].describe()

count       556.000000
mean     172150.250000
std      115735.338314
min        9089.000000
25%       85872.250000
50%      163414.000000
75%      216626.500000
max      501837.000000
Name: code, dtype: float64